In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import open3d as o3d

In [ ]:
#todo: set root dir
root_dir = ""

img_ind = [5095, 5324, 5334, 5349, 5442, 5449, 5632, 5641, 5722, 5879, 5902, 6195, 6299, 6335, 6382, 6412]
rgbs = []
depths = []
#loading of rgbs and depth maps
for i in img_ind:
    rgb = cv2.imread(os.path.join(root_dir, "3d/rgb/", "img_" + str(i) + ".png"))
    rgb = cv2.cvtColor(rgb, cv2.COLOR_BGR2RGB)
    rgbs.append(rgb)
    depth = cv2.imread(os.path.join(root_dir, "3d/depth/", "img_" + str(i) + ".png"), -1)
    depth = np.asarray(depth, dtype=np.float32)
    depths.append(depth / 1000.0)

In [ ]:
#visualize images using matplotlib
fig = plt.figure(figsize=(25.00, 10.0))
for i in range(0, len(rgbs)):
    plt.subplot(4, 8, i + (i // 8) * 8 + 1)
    plt.axis("off")
    plt.imshow(rgbs[i])
    plt.subplot(4, 8, i + ((i // 8 + 1) * 8) + 1)
    plt.axis("off")
    plt.imshow(1. / depths[i], cmap="plasma")


In [ ]:
fx = 5.1885790117450188e+02
fy = 5.1946961112127485e+02
cx = 3.2558244941119034e+02 - 40
cy = 2.5373616633400465e+02 - 44

#create intrinsic matrix and its inverse
K = np.eye(3)
K[0, 0] = fx
K[0, 2] = cx
K[1, 1] = fy
K[1, 2] = cy

K_inv = np.linalg.inv(K)


In [ ]:
#form a set of points for images 3 x Npixels: (x, y, 1), reshape depths to (h * w), unprojection part and coloring part
h, w, _ = rgbs[0].shape
xv = np.linspace(0, w - 1, w)
yv = np.linspace(0, h - 1, h)
xx, yy = np.meshgrid(xv, yv)
#
points2d = np.stack([xx, yy, np.ones_like(xx)], axis=2)
points2d = np.reshape(points2d, (h * w, 3))
points2d = np.transpose(points2d)

points3D_unscaled = np.matmul(K_inv, points2d)
#print(points3D_unscaled)
points3D = []
for i in range(0, len(depths)):
    depth_flattened_i = np.reshape(depths[i], (h * w))
    points3D.append(np.transpose(points3D_unscaled * depth_flattened_i))

print(points3D[0].shape)    
points3D_colored = []
for i in range(0, len(rgbs)):
   color_flattened_i = np.reshape(rgbs[i], (h * w, 3))
   points3D_colored.append(np.concatenate([points3D[i], color_flattened_i], axis=1))

#print(points3D_colored[0][0])

In [ ]:
#visualize point cloud(s)

cloud = o3d.geometry.PointCloud()
index2vis = 1
cloud.points = o3d.utility.Vector3dVector(points3D_colored[index2vis][:, :3])
cloud.colors = o3d.utility.Vector3dVector(points3D_colored[index2vis][:, 3:] / 255.0)

o3d.visualization.draw_geometries([cloud])

In [ ]:
#transform and project back, using K (play with the focal length of K)
#create 3D transform matrix (4x4, or 3x4), lets say with z translation -1.0
M = np.eye(4)[:-1, :]
#modify to add translation?
M[2, 3] = -0.3
M[0, 3] = 0.4

#go to projected homogeneous (but keep Z)
points2d_new = []
for i in range(0, len(points3D)):
    points3D_hom_i = np.concatenate([points3D[i], np.ones((h*w, 1), dtype=np.float32)], axis=1)
    points3D_hom_i = np.transpose(points3D_hom_i)
    transformed_points_i = np.matmul(M, points3D_hom_i)
    projected_points_i = np.matmul(K, transformed_points_i)
    projected_points_i[:2] = projected_points_i[:2] / projected_points_i[2]
    points2d_new.append(projected_points_i)

In [ ]:
images_new = np.zeros((len(points2d_new), h, w, 4), dtype=np.uint8)
for i in range(0, len(points2d_new)):
   print("processed image " + str(i))
   p_new_i = np.transpose(points2d_new[i])
   color_flattened_i = np.reshape(rgbs[i], (h * w, 3))
   for j in range(0, p_new_i.shape[0]):
       px = int(p_new_i[j][0])
       py = int(p_new_i[j][1])
       z = p_new_i[j][2]
       if px >= 0 and px < w and py >= 0 and py < h and (images_new[i][py, px, 3] < 0.001 or images_new[i][py, px, 3] > z):
           images_new[i][py, px, 3] = z
           images_new[i][py, px, :3] = color_flattened_i[j]

In [ ]:
fig = plt.figure(figsize=(30, 25.0))
for i in range(0, len(rgbs)):
    plt.subplot(4, 4, i + 1)
    plt.axis("off")
    plt.imshow(images_new[i][:, :, :3])